<a href="https://colab.research.google.com/github/AhmedZaghl0ul/Machine-Learning/blob/main/Lung_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/DataSets/

/content/drive/MyDrive/DataSets


In [3]:
dir = 'lung_cancer.csv'

In [4]:
import pandas as pd
df = pd.read_csv(dir)

In [5]:
class_names = ['No' , 'Yes']

In [6]:
df.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 309 non-null    object
 1   AGE                    309 non-null    int64 
 2   SMOKING                309 non-null    int64 
 3   YELLOW_FINGERS         309 non-null    int64 
 4   ANXIETY                309 non-null    int64 
 5   PEER_PRESSURE          309 non-null    int64 
 6   CHRONIC DISEASE        309 non-null    int64 
 7   FATIGUE                309 non-null    int64 
 8   ALLERGY                309 non-null    int64 
 9   WHEEZING               309 non-null    int64 
 10  ALCOHOL CONSUMING      309 non-null    int64 
 11  COUGHING               309 non-null    int64 
 12  SHORTNESS OF BREATH    309 non-null    int64 
 13  SWALLOWING DIFFICULTY  309 non-null    int64 
 14  CHEST PAIN             309 non-null    int64 
 15  LUNG_CANCER            

In [9]:
df.duplicated().sum()

np.int64(33)

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.shape

(276, 16)

In [12]:
df.columns

Index(['GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY',
       'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
       'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
       'SWALLOWING DIFFICULTY', 'CHEST PAIN', 'LUNG_CANCER'],
      dtype='object')

In [14]:
df['LUNG_CANCER'].value_counts()

,count
LUNG_CANCER,
YES,238
NO,38


In [15]:
x = df.drop('LUNG_CANCER' , axis=1)
y = df['LUNG_CANCER']

In [16]:
print(x.shape , y.shape)

(276, 15) (276,)


In [44]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x['GENDER'] = le.fit_transform(x['GENDER'])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [45]:
y = y.map({'YES':1 , 'NO':0})

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [46]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.2 , random_state=42)

In [47]:
print(x_train.shape , x_test.shape)

(220, 15) (56, 15)


In [48]:
from collections import Counter
counter = Counter(y_train)
print(counter)

Counter({1: 194, 0: 26})


In [49]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
x_train , y_train = ros.fit_resample(x_train , y_train)

In [50]:
print(x_train.shape , x_test.shape)

(388, 15) (56, 15)


In [51]:
from collections import Counter
counter = Counter(y_train)
print(counter)

Counter({1: 194, 0: 194})


In [52]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

param_grid = {'C': [0.001 ,0.1 ,0.5, 1, 10,15,50, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear']}

grid = GridSearchCV(SVC(), param_grid,cv = 5)
grid.fit(x_train, y_train)

print("Best parameters found: ", grid.best_params_)
print("Best score found: ", grid.best_score_)

Best parameters found:  {'C': 0.5, 'gamma': 1, 'kernel': 'rbf'}
Best score found:  0.9896769896769897


In [53]:
model = grid.best_estimator_
ypred = model.predict(x_test)


print(classification_report(y_test, ypred,digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.2500    0.4000        12
           1     0.8302    1.0000    0.9072        44

    accuracy                         0.8393        56
   macro avg     0.9151    0.6250    0.6536        56
weighted avg     0.8666    0.8393    0.7985        56



In [54]:
from sklearn.tree import DecisionTreeClassifier

param_grid_dt = {'max_depth': [None, 5, 10, 15, 20],
                 'min_samples_split': [2, 5, 10],
                 'min_samples_leaf': [1, 2, 4],
                 'criterion': ['gini', 'entropy']}

grid_dt = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv=5, verbose=2)
grid_dt.fit(x_train, y_train)

print("Best parameters found for Decision Tree: ", grid_dt.best_params_)
print("Best score found for Decision Tree: ", grid_dt.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_

In [55]:
model_dt = grid_dt.best_estimator_
ypred_dt = model_dt.predict(x_test)

print(classification_report(y_test, ypred_dt,digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.5000    0.6667        12
           1     0.8800    1.0000    0.9362        44

    accuracy                         0.8929        56
   macro avg     0.9400    0.7500    0.8014        56
weighted avg     0.9057    0.8929    0.8784        56



In [56]:
from sklearn.ensemble import RandomForestClassifier

param_grid_rf = {'n_estimators': [100, 200, 500],
                 'max_depth': [None, 5, 10, 15],
                 'min_samples_split': [2, 5, 10],
                 'min_samples_leaf': [1, 2, 4],
                 'criterion': ['gini', 'entropy']}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, verbose=2, n_jobs=-1)
grid_rf.fit(x_train, y_train)

print("Best parameters found for Random Forest: ", grid_rf.best_params_)
print("Best score found for Random Forest: ", grid_rf.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters found for Random Forest:  {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best score found for Random Forest:  0.9664335664335665


In [57]:
model_rf = grid_rf.best_estimator_
ypred_rf = model_rf.predict(x_test)

print(classification_report(y_test, ypred_rf,digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.5833    0.7368        12
           1     0.8980    1.0000    0.9462        44

    accuracy                         0.9107        56
   macro avg     0.9490    0.7917    0.8415        56
weighted avg     0.9198    0.9107    0.9014        56



In [58]:
!pip install xgboost

In [59]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

param_grid_xgb = {
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

grid_xgb = GridSearchCV(XGBClassifier(random_state=42), param_grid_xgb, cv=5, verbose=2, n_jobs=-1)
grid_xgb.fit(x_train, y_train)

print("Best parameters found for XGBoost: ", grid_xgb.best_params_)
print("Best score found for XGBoost: ", grid_xgb.best_score_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best parameters found for XGBoost:  {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.6}
Best score found for XGBoost:  0.9587745587745587


In [60]:
model_xgb = grid_xgb.best_estimator_
ypred_xgb = model_xgb.predict(x_test)

print(classification_report(y_test, ypred_xgb,digits=4))

              precision    recall  f1-score   support

           0     0.8000    0.6667    0.7273        12
           1     0.9130    0.9545    0.9333        44

    accuracy                         0.8929        56
   macro avg     0.8565    0.8106    0.8303        56
weighted avg     0.8888    0.8929    0.8892        56

